In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract




tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
import cv2
import pytesseract
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [4]:
#My custom class for declare region of interest.
#from drive.MyDrive.python_lib_data import ImageConstantROI

class CCCD(object):
    ROIS = {
        "id": [(270, 120, 620, 80)],
        "name": [
            (290, 183, 500, 64),
        ],
        "birth_date": [(320, 241, 500, 48)],
        "gender": [(320, 280, 280, 47)],
        "address": [
            (320, 317, 400, 47),
            (320, 351, 200, 47),
            (320, 390, 400, 47),
            (320, 430, 350, 43)
        ]
    }

    CHECK_ROI = [(313, 174, 597, 63)]

CCCD.ROIS['id'][0]

(270, 120, 620, 80)

In [5]:
#Custom function to show open cv image on notebook.
def display_img(cvImg):
    cvImg = cv2.cvtColor(cvImg, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10,8))
    plt.imshow(cvImg)
    plt.axis('off')
    plt.show()

In [6]:
#Loading image using cv2
baseImg = cv2.imread('drive/MyDrive/python_lib_data/sample_data/ktp1.jpg')

#Declare image size, width height and chanel
baseH, baseW, baseC = baseImg.shape

display_img(baseImg)

[ WARN:0@0.547] global loadsave.cpp:248 findDecoder imread_('drive/MyDrive/python_lib_data/sample_data/ktp1.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
#Create a custom function to cropped image base on religion of interest
def cropImageRoi(image, roi):
    roi_cropped = image[
        int(roi[1]) : int(roi[1] + roi[3]), int(roi[0]) : int(roi[0] + roi[2])
    ]
    return roi_cropped

testCrop = cropImageRoi(baseImg, CCCD.ROIS['id'][0] )
display_img(testCrop)

In [ ]:
testCrop3 = cropImageRoi(baseImg, CCCD.ROIS['name'][0] )
display_img(testCrop3)

In [ ]:
testCrop6 = cropImageRoi(baseImg, CCCD.ROIS['gender'][0] )
display_img(testCrop6)

In [ ]:
testCrop2 = cropImageRoi(baseImg, CCCD.ROIS['birth_date'][0] )
display_img(testCrop2)

In [ ]:
testCrop7 = cropImageRoi(baseImg, CCCD.ROIS['address'][0] )
display_img(testCrop7)

testCrop8 = cropImageRoi(baseImg, CCCD.ROIS['address'][1] )
display_img(testCrop8)

testCrop8 = cropImageRoi(baseImg, CCCD.ROIS['address'][2] )
display_img(testCrop8)

testCrop8 = cropImageRoi(baseImg, CCCD.ROIS['address'][3] )
display_img(testCrop8)

In [ ]:
#Extract data from cropped image
pytesseract.image_to_string(testCrop, config='--oem 1 --psm 6')

In [ ]:
def preprocessing_image(img):
    #Resize image
    #img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

    #convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.multiply(gray, 1.5)

    #blur remove noise
    blured1 = cv2.medianBlur(gray,3)
    blured2 = cv2.medianBlur(gray,51)
    divided = np.ma.divide(blured1, blured2).data
    normed = np.uint8(255*divided/divided.max())


    #Threshold image
    th, threshed = cv2.threshold(normed, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)


    return threshed

testPreprocess = preprocessing_image(testCrop)
display_img(testPreprocess)

In [ ]:
#Extract data from cropped image
pytesseract.image_to_string(testCrop, config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')

In [ ]:
#Extract all interest data from image
MODEL_CONFIG = '--oem 3 --psm 10'
for key, roi in CCCD.ROIS.items():
    data = ''
    for r in roi:
        crop_img = cropImageRoi(baseImg, r)

        #For a small pxi image only has number, do not preprocessing it is better
        if key != 'date_expire':
            crop_img = preprocessing_image(crop_img)

        display_img(crop_img)


        data += pytesseract.image_to_string(crop_img, config = MODEL_CONFIG) + ' '

    print(f"{key} : {data.strip()}")

In [ ]:
#Let create a function to reuse it later
def extractDataFromIdCard(img):
    for key, roi in CCCD.ROIS.items():
        data = ''
        for r in roi:
            crop_img = cropImageRoi(img, r)

            #For a small pxi image only has number, do not preprocessing it is better
            if key != 'date_expire':
                crop_img = preprocessing_image(crop_img)

            display_img(crop_img)

            #Extract data from image using pytesseract
            data += pytesseract.image_to_string(crop_img, config = MODEL_CONFIG) + ' '

        print(f"{key} : {data.strip()}")

In [ ]:
#Load image ktp yg gambar-nya miring
img2 = cv2.imread('drive/MyDrive/python_lib_data/sample_data/ktp3.jpg')

display_img(img2)

In [ ]:
#Init orb, keypoints detection on base Image
orb = cv2.ORB_create(1000)

kp, des = orb.detectAndCompute(baseImg, None)
imgKp = cv2.drawKeypoints(baseImg,kp, None)

display_img(imgKp)

In [ ]:
PER_MATCH = 0.25

#Detect keypoint on img2
kp1, des1 = orb.detectAndCompute(img2, None)

#Init BF Matcher, find the matches points of two images
bf = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = list(bf.match(des1, des))

#Select top 30% best matcher
matches.sort(key=lambda x: x.distance)
best_matches = matches[:int(len(matches)*PER_MATCH)]

#Show match img
imgMatch = cv2.drawMatches(img2, kp1, baseImg, kp, best_matches,None, flags=2)
display_img(imgMatch)

In [ ]:
#Init source points and destination points for findHomography function.
srcPoints = np.float32([kp1[m.queryIdx].pt for m in best_matches]).reshape(-1,1,2)
dstPoints = np.float32([kp[m.trainIdx].pt for m in best_matches]).reshape(-1,1,2)


#Find Homography of two images
matrix_relationship, _ = cv2.findHomography(srcPoints, dstPoints,cv2.RANSAC, 5.0)

#Transform the image to have the same structure as the base image
img_final = cv2.warpPerspective(img2, matrix_relationship, (baseW, baseH))

display_img(img_final)

In [ ]:
extractDataFromIdCard(img_final)